## Loading Packages

In [3]:
!pip3 install torch==1.9.1+cu102 torchvision==0.10.1+cu102 torchaudio===0.9.1 -f https://download.pytorch.org/whl/torch_stable.html

You should consider upgrading via the 'c:\users\ajv13\anaconda3\python.exe -m pip install --upgrade pip' command.


In [1]:
import os
import cv2
import torch
import torchvision
import numpy as np
import pandas as pd
import torch.utils.data as data

In [2]:
from processing_functions import *

## Making Data and Data Loader

### _Smote_

In [ ]:
class TeethDataLoader_wSMOTE(data.Dataset): 
    
    def __init__(self, train_dataloader):
        self.images, self.labels = SMOTE_Balance(train_dataloader)
           
    def __len__(self):
        return len(self.images)

    def __getitem__(self, index):
        img = self.images[index]
        label = self.labels[index]               # class # 
        
        return img.float(), torch.FloatTensor([label])# returns tensor of modified image and label

### _Upsampling_

In [ ]:
class TeethDataLoader_Simplified(data.Dataset):
    
    def __init__(self, path_to_df, included_anomalies):
        df = pd.read_csv(path_to_df)
        df = df[df['anomaly_code'].isin(include_anomalies)]
        self.images, self.labels = df['file_paths'], df['anomaly_code']
        
    def __len__(self):
        return len(self.images)

    def __getitem__(self, index):
        img = cv2.imread(self.images[index])     # loads image from file
        img = cv2.resize(img, (224, 224))        # resize image to 224x224
        img = np.array(img, dtype=np.float32)    # change data type to float
        img = (img / 255.)                       # normalize colors to be 0-1
        img = img[:, :, (2, 1, 0)]               # reorder RGB
        img = torchvision.transforms.ToTensor()(img)  # convert to tensor
        label = self.labels[index]               # 1 or 0
        
        return img.float(), torch.FloatTensor([label]) # returns tensor of modified image and label
        
   

## Old Loader

In [ ]:
class TeethDataLoader(data.Dataset): # input is folder name where folders of teeth are stored
                                        # requires overwrite __getitem__(), supporting fetching a data sample for a given key
                                        # can also overwrite __len__(), which is expected to return the size of the dataset

    
    def __init__(self, path_to_df): # path below = ./training_images/
                
        xray_filenames = os.listdir(xray_path)
        anomaly_filenames = os.listdir(anomaly_path)
        segmentation_filenames = os.listdir(segmentation_path)

        output_path = 'SegmentedTeethImages/'

        for i in range(len(anomaly_filenames)):
            # Create dataframe with tooth number
            anomalies_df = anomaly_matching(anomaly_path + anomaly_filenames[i], 
                                            segmentation_path + segmentation_filenames[i], 
                                            io.imread(xray_path + xray_filenames[i]).shape, 
                                            normalized=True)
            anomalies_df = remove_duplicates(anomalies_df)
            #anomalies_df = anomalies_df[(anomalies_df['anomaly_category'] == 0) | (anomalies_df['anomaly_category'] == 1)]
            
            for index, row in anomalies_df.iterrows():
                yolo_coord = row[['x_center', 'y_center', 'width', 'height']].to_list()
                tooth_file = extract_image(xray_path + xray_filenames[i],
                                           yolo_coord, 
                                           int(row['tooth_number']), 
                                           output_folder=output_path, 
                                           print_names=False)
                self.images.append(tooth_file)
                self.labels.append(row['anomaly_category'])
                
                if add_rotation:
                    rotated_images = image_rotation(xray_path + xray_filenames[i], deg=rotation_deg,
                                                    output_folder=output_path, print_names=False)
                    rotated_labels = row['anomaly_category'] * len(rotated_images)
                    self.images.extend(rotated_images)
                    self.labels.extend(rotated_labels)
                
                if add_flip:
                    flipped_images = image_flip(xray_path + xray_filenames[i], output_folder=output_path, print_names=False)
                    flipped_labels = row['anomaly_category'] * len(flipped_images)
                    self.images.extend(flipped_images)
                    self.labels.extend(flipped_labels)
                
                if add_noise:
                    noise_image = image_noise(xray_path + xray_filenames[i], sigma=.1,
                                              output_folder=output_path, print_names=False)
                    noise_label = row['anomaly_category']
                    self.images.extend(noise_image)
                    self.labels.extend(noise_label)
                    
                if add_blur:
                    blur_image = image_gauss_blur(xray_path + xray_filenames[i], sigma=1,
                                                  output_folder=output_path, print_names=False)
                    blur_label = row['anomaly_category']
                    self.images.extend(noise_image)
                    self.labels.extend(noise_label)

    def __len__(self):
        return len(self.images)

    def __getitem__(self, index):
        img = cv2.imread(self.images[index])     # loads image from file
        img = cv2.resize(img, (224, 224))        # resize image to 224x224
        img = np.array(img, dtype=np.float32)    # change data type to float
        img = (img / 255.)                       # normalize colors to be 0-1
        img = img[:, :, (2, 1, 0)]               # reorder RGB
        img = torchvision.transforms.ToTensor()(img)  # convert to tensor
        label = self.labels[index]               # 1 or 0
        
        return img.float(), torch.FloatTensor([label]) # returns tensor of modified image and label

## Model

In [98]:
import math
import torch.nn as nn
import torch.nn.functional as F

In [98]:
class BasicConv(nn.Module):
    # initialize class
    def __init__(self, in_planes, out_planes, kernel_size, stride=1, 
                 padding=0, dilation=1, groups=1, relu=True, bn=True, bias=False):
        super(BasicConv, self).__init__()
        self.out_channels = out_planes
        # def conv layer, in_planes/out_planes = size of features
        self.conv = nn.Conv2d(in_planes, out_planes, 
                              kernel_size=kernel_size, stride=stride, 
                              padding=padding, dilation=dilation,
                              groups=groups, bias=bias)
        # batch normalization - normalization of the layers' inputs by re-centering and re-scaling
        self.bn = nn.BatchNorm2d(out_planes,eps=1e-5, momentum=0.01, affine=True) if bn else None
        # ReLu activation
        self.relu = nn.ReLU() if relu else None

    # create feed-forward network for conv layer
    def forward(self, x):
        x = self.conv(x)           # only 1 layer
        if self.bn is not None:
            x = self.bn(x)
        if self.relu is not None:
            x = self.relu(x)
        return x

    
    
class Flatten(nn.Module):
    def forward(self, x):
        return x.view(x.size(0), -1) # reshape to flatten tensor which is necessary in order to pass data into a linear layer
                                     # add -1
                                     # no flatten function in pytorch so need to create it

    
    
class ChannelGate(nn.Module):
    def __init__(self, gate_channels, reduction_ratio=16, pool_types=['avg', 'max']):
        super(ChannelGate, self).__init__()
        self.gate_channels = gate_channels
        self.mlp = nn.Sequential(
            Flatten(),
            nn.Linear(gate_channels, gate_channels // reduction_ratio),
            nn.ReLU(),
            nn.Linear(gate_channels // reduction_ratio, gate_channels)
            )
        self.pool_types = pool_types
    def forward(self, x):
        channel_att_sum = None
        for pool_type in self.pool_types:
            if pool_type=='avg':
                avg_pool = F.avg_pool2d( x, (x.size(2), x.size(3)), stride=(x.size(2), x.size(3)))
                channel_att_raw = self.mlp( avg_pool )
            elif pool_type=='max':
                max_pool = F.max_pool2d( x, (x.size(2), x.size(3)), stride=(x.size(2), x.size(3)))
                channel_att_raw = self.mlp( max_pool )
            elif pool_type=='lp':
                lp_pool = F.lp_pool2d( x, 2, (x.size(2), x.size(3)), stride=(x.size(2), x.size(3)))
                channel_att_raw = self.mlp( lp_pool )
            elif pool_type=='lse':
                # LSE pool only
                lse_pool = logsumexp_2d(x)
                channel_att_raw = self.mlp( lse_pool )

            if channel_att_sum is None:
                channel_att_sum = channel_att_raw
            else:
                channel_att_sum = channel_att_sum + channel_att_raw

        scale = F.sigmoid( channel_att_sum ).unsqueeze(2).unsqueeze(3).expand_as(x)
        return x * scale

    
    
def logsumexp_2d(tensor):
    tensor_flatten = tensor.view(tensor.size(0), tensor.size(1), -1)
    s, _ = torch.max(tensor_flatten, dim=2, keepdim=True)
    outputs = s + (tensor_flatten - s).exp().sum(dim=2, keepdim=True).log()
    return outputs



class ChannelPool(nn.Module):
    def forward(self, x):
        return torch.cat( (torch.max(x,1)[0].unsqueeze(1), torch.mean(x,1).unsqueeze(1)), dim=1 )

    
    
class SpatialGate(nn.Module):
    def __init__(self):
        super(SpatialGate, self).__init__()
        kernel_size = 7
        self.compress = ChannelPool()
        self.spatial = BasicConv(2, 1, kernel_size, stride=1, padding=(kernel_size-1) // 2, relu=False)
    def forward(self, x):
        x_compress = self.compress(x)
        x_out = self.spatial(x_compress)
        scale = F.sigmoid(x_out) # broadcasting
        return x * scale

    
    
class CBAM(nn.Module):
    def __init__(self, gate_channels, reduction_ratio=16, pool_types=['avg', 'max'], no_spatial=False):
        super(CBAM, self).__init__()
        self.ChannelGate = ChannelGate(gate_channels, reduction_ratio, pool_types)
        self.no_spatial=no_spatial
        if not no_spatial:
            self.SpatialGate = SpatialGate()
    def forward(self, x):
        x_out = self.ChannelGate(x)
        if not self.no_spatial:
            x_out = self.SpatialGate(x_out)
        return x_out

In [116]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import math
from torch.nn import init

def conv3x3(in_planes, out_planes, stride=1):
    "3x3 convolution with padding"
    return nn.Conv2d(in_planes, out_planes, kernel_size=3, stride=stride,
                     padding=1, bias=False)

class BasicBlock(nn.Module):
    expansion = 1

    def __init__(self, inplanes, planes, stride=1, downsample=None, use_cbam=False):
        super(BasicBlock, self).__init__()
        self.conv1 = conv3x3(inplanes, planes, stride)
        self.bn1 = nn.BatchNorm2d(planes)
        self.relu = nn.ReLU(inplace=True)
        self.conv2 = conv3x3(planes, planes)
        self.bn2 = nn.BatchNorm2d(planes)
        self.downsample = downsample
        self.stride = stride

        if use_cbam:
            self.cbam = CBAM( planes, 16 )
        else:
            self.cbam = None

    def forward(self, x):
        residual = x

        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)

        out = self.conv2(out)
        out = self.bn2(out)

        if self.downsample is not None:
            residual = self.downsample(x)

        if not self.cbam is None:
            out = self.cbam(out)

        out += residual
        out = self.relu(out)

        return out

class Bottleneck(nn.Module):
    expansion = 4

    def __init__(self, inplanes, planes, stride=1, downsample=None, use_cbam=False):
        super(Bottleneck, self).__init__()
        self.conv1 = nn.Conv2d(inplanes, planes, kernel_size=1, bias=False)
        self.bn1 = nn.BatchNorm2d(planes)
        self.conv2 = nn.Conv2d(planes, planes, kernel_size=3, stride=stride,
                               padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(planes)
        self.conv3 = nn.Conv2d(planes, planes * 4, kernel_size=1, bias=False)
        self.bn3 = nn.BatchNorm2d(planes * 4)
        self.relu = nn.ReLU(inplace=True)
        self.downsample = downsample
        self.stride = stride

        if use_cbam:
            self.cbam = CBAM( planes * 4, 16 )
        else:
            self.cbam = None

    def forward(self, x):
        residual = x

        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)

        out = self.conv2(out)
        out = self.bn2(out)
        out = self.relu(out)

        out = self.conv3(out)
        out = self.bn3(out)

        if self.downsample is not None:
            residual = self.downsample(x)

        if not self.cbam is None:
            out = self.cbam(out)

        out += residual
        out = self.relu(out)

        return out

class ResNet(nn.Module):
    def __init__(self, block, layers,  network_type, num_classes, att_type=None):
        self.inplanes = 64
        super(ResNet, self).__init__()

        self.network_type = network_type
        # different model config between ImageNet and CIFAR 
        if network_type == "ImageNet":
            self.conv1 = nn.Conv2d(3, 64, kernel_size=7, stride=2, padding=3, bias=False)
            self.maxpool = nn.MaxPool2d(kernel_size=3, stride=2, padding=1)
            self.avgpool = nn.AvgPool2d(7)
        else:
            self.conv1 = nn.Conv2d(3, 64, kernel_size=3, stride=1, padding=1, bias=False)

        self.bn1 = nn.BatchNorm2d(64)
        self.relu = nn.ReLU(inplace=True)

        if att_type=='BAM':
            self.bam1 = BAM(64*block.expansion)
            self.bam2 = BAM(128*block.expansion)
            self.bam3 = BAM(256*block.expansion)
        else:
            self.bam1, self.bam2, self.bam3 = None, None, None

        self.layer1 = self._make_layer(block, 64,  layers[0], att_type=att_type)
        self.layer2 = self._make_layer(block, 128, layers[1], stride=2, att_type=att_type)
        self.layer3 = self._make_layer(block, 256, layers[2], stride=2, att_type=att_type)
        self.layer4 = self._make_layer(block, 512, layers[3], stride=2, att_type=att_type)

        self.fc = nn.Linear(512 * block.expansion, num_classes)
        #self.fc = nn.Linear(512 * block.expansion, 1) # linear is output of probability distributions
        self.softmax = torch.nn.Sigmoid()
        init.kaiming_normal(self.fc.weight)
        for key in self.state_dict():
            if key.split('.')[-1]=="weight":
                if "conv" in key:
                    init.kaiming_normal(self.state_dict()[key], mode='fan_out')
                if "bn" in key:
                    if "SpatialGate" in key:
                        self.state_dict()[key][...] = 0
                    else:
                        self.state_dict()[key][...] = 1
            elif key.split(".")[-1]=='bias':
                self.state_dict()[key][...] = 0

    def _make_layer(self, block, planes, blocks, stride=1, att_type=None):
        downsample = None
        if stride != 1 or self.inplanes != planes * block.expansion:
            downsample = nn.Sequential(
                nn.Conv2d(self.inplanes, planes * block.expansion,
                          kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(planes * block.expansion),
            )

        layers = []
        layers.append(block(self.inplanes, planes, stride, downsample, use_cbam=att_type=='CBAM'))
        self.inplanes = planes * block.expansion
        for i in range(1, blocks):
            layers.append(block(self.inplanes, planes, use_cbam=att_type=='CBAM'))

        return nn.Sequential(*layers)

    def forward(self, x):
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu(x)
        if self.network_type == "ImageNet":
            x = self.maxpool(x)

        x = self.layer1(x)
        if not self.bam1 is None:
            x = self.bam1(x)

        x = self.layer2(x)
        if not self.bam2 is None:
            x = self.bam2(x)

        x = self.layer3(x)
        if not self.bam3 is None:
            x = self.bam3(x)

        x = self.layer4(x)

        if self.network_type == "ImageNet":
            x = self.avgpool(x)
        else:
            x = F.avg_pool2d(x, 4)
            
        x = x.view(x.size(0), -1)
        x = self.fc(x)

        #return F.normalize(x, dim=-1)
        return self.softmax(x)
def ResidualNet(network_type, depth, num_classes, att_type):

    assert network_type in ["ImageNet", "CIFAR10", "CIFAR100"], "network type should be ImageNet or CIFAR10 / CIFAR100"
    assert depth in [5, 18, 34, 50, 101], 'network depth should be 18, 34, 50 or 101'

    if depth == 18:
        model = ResNet(BasicBlock, [2, 2, 2, 2], network_type, num_classes, att_type)
        
    elif depth == 5:
        model = ResNet(BasicBlock, [1, 1, 2, 1], network_type, num_classes, att_type)

    elif depth == 34:
        model = ResNet(BasicBlock, [3, 4, 6, 3], network_type, num_classes, att_type)

    elif depth == 50:
        model = ResNet(Bottleneck, [3, 4, 6, 3], network_type, num_classes, att_type)

    elif depth == 101:
        model = ResNet(Bottleneck, [3, 4, 23, 3], network_type, num_classes, att_type)

    return model

# Training

In [100]:
import torch
import torch.nn as nn
import torch.optim as optim
from tqdm import tqdm
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

Change confusion matrix for model 1

In [118]:
best_accuracy = 0

def train(train_loader, model, classes, optimizer, epoch):
    model.train()
    running_mse = 0
    batch_idx = 0
    correct, total = 0, 0
    confusion_matrix = torch.zeros(classes, classes)
 
    pd, gt = [], []
    for (x, y) in tqdm(train_loader):
        batch_idx += 1
        x = x.to(device) # move data to GPU
        y = y.to(device) # move data to GPU

        out = model(x)
        criterion = nn.CrossEntropyLoss()
        #criterion = nn.BCELoss()                    # binary cross entropy loss
        loss = criterion(out, y.squeeze(1).long())
        #loss = criterion(out.squeeze(1), y.squeeze(1)) # calculate loss based on pred output vs actuals y, 
                                                       # .squeeze removes dimensions of 1
        running_mse += loss.item() 
        optimizer.zero_grad()        # start gradient at 0
        loss.backward()
        optimizer.step()

        _, predicted = torch.max(out.data, 1)
        #predicted = (out.data > 0.5).int()

        for p in predicted:
            pd.append(p)
        for g in y.squeeze(1):
            gt.append(g)

        total += y.squeeze(1).size(0)
        correct += (predicted == y.squeeze(1)).sum().item()
        for t, p in zip(y.squeeze(1).view(-1), predicted.view(-1)):
            confusion_matrix[t.long(), p.long()] += 1
    
    pd = torch.as_tensor(pd).numpy()
    gt = torch.as_tensor(gt).numpy()

    running_mse = running_mse / batch_idx
    print('Epoch %d, loss = %.4f, batch_idx= %d' % (epoch, running_mse, batch_idx))
    print('Epoch: %d Accuracy of the Train Images: %f' %(epoch, 100 * correct / total))
    print('Confusion Matrix', np.round(confusion_matrix.cpu().numpy(), 2))
    print('Classification Report', classification_report(gt, pd))
    

def valid(valid_loader, model, classes, epoch):
    global best_accuracy
    model.eval()
    correct, total = 0, 0
    running_mse = 0
    confusion_matrix = torch.zeros(classes, classes)
    pd, gt = [], []
    with torch.no_grad():
        for x, y in tqdm(valid_loader):
            x = x.to(device)
            y = y.to(device)

            out = model(x) # add .to(device) ???
            criterion = nn.CrossEntropyLoss()
            #criterion = nn.BCELoss()
            loss = criterion(out, y.squeeze(1).long())
            #loss = criterion(out.squeeze(1), y.squeeze(1))
            running_mse += loss.item()
            _, predicted = torch.max(out.data, 1)
            #predicted = (out.data > 0.5).int()

            for p in predicted:
                pd.append(p)
            for g in y.squeeze(1):
                gt.append(g)

            total += y.squeeze(1).size(0)
            correct += (predicted == y.squeeze(1)).sum().item()
            for t, p in zip(y.squeeze(1).view(-1), predicted.view(-1)):
                confusion_matrix[t.long(), p.long()] += 1

    if best_accuracy < 100 * (correct/total):
        best_accuracy = 100 * (correct/total)
        
    pd = torch.as_tensor(pd).numpy()
    gt = torch.as_tensor(gt).numpy()
      
    print('Epoch: %d Accuracy of the Valid Images: %f' %(epoch, 100 * correct / total))
    print('Confusion Matrix', np.round(confusion_matrix.cpu().numpy(), 2))
    print('Classification Report', classification_report(gt, pd))

### GPU

In [ ]:
# check if GPU available and assign it
torch.cuda.is_available()
device = torch.device('cuda',1)
device

#### _Model 1_

In [ ]:
epochs = 50

model = ResidualNet('ImageNet', 5, 8, 'CBAM')
model.to(device) # sends model to GPU
optimizer = optim.Adam(model.parameters(), lr=0.00003)

path_to_df =

batch_size = 64
epochs = 100
dataset = TeethDataLoader_simplified(path_to_df, [0,1, 2, 3, 4, 5, 6, 7])


# generate shuffled sequence of numbers based on how many images, split train/test
train_idx, valid_idx= train_test_split(np.arange(len(dataset.labels)), test_size=0.2, shuffle=True, stratify=dataset.labels)

# Samples elements randomly from a given list of indices, without replacement
train_sampler = torch.utils.data.SubsetRandomSampler(train_idx)
valid_sampler = torch.utils.data.SubsetRandomSampler(valid_idx)

# loads images associated with ids/samples from above
train_loader = torch.utils.data.DataLoader(dataset, batch_size=batch_size, sampler=train_sampler)
valid_loader = torch.utils.data.DataLoader(dataset, batch_size=batch_size, sampler=valid_sampler)

#### _*smote_

In [ ]:
dataset_smote = TeethDataLoader_wSMOTE(train_loader)
train_loader_smote = torch.utils.data.DataLoader(dataset_smote, batch_size=batch_size)

In [ ]:
for epoch in range(1, epoch+1):
    train(train_loader_smote, model, optimizer, epoch)
    valid(valid_loader, model, epoch)

#### _Model 2_

In [ ]:
epochs = 50

model2 = ResidualNet('ImageNet', 5, 9, 'CBAM')
model2.to(device) # sends model to GPU
optimizer = optim.Adam(model2.parameters(), lr=0.00003)

path_to_df =

batch_size = 64
epochs = 100
dataset2 = TeethDataLoader_simplified(path_to_df, [0,1, 2, 3, 4, 5, 6, 7, 8])

# generate shuffled sequence of numbers based on how many images, split train/test
train_idx, valid_idx= train_test_split(np.arange(len(dataset2.labels)), test_size=0.2, shuffle=True, stratify=dataset.labels)

# Samples elements randomly from a given list of indices, without replacement
train_sampler = torch.utils.data.SubsetRandomSampler(train_idx)
valid_sampler = torch.utils.data.SubsetRandomSampler(valid_idx)

# loads images associated with ids/samples from above
train_loader = torch.utils.data.DataLoader(dataset2, batch_size=batch_size, sampler=train_sampler)
valid_loader = torch.utils.data.DataLoader(dataset2, batch_size=batch_size, sampler=valid_sampler)

#### _*smote_

In [ ]:
dataset_smote = TeethDataLoader_wSMOTE(train_loader)
train_loader_smote = torch.utils.data.DataLoader(dataset_smote, batch_size=batch_size)

In [ ]:
for epoch in range(1, epoch+1):
    train(train_loader_smote, model, optimizer, epoch)
    valid(valid_loader, model, epoch)

# EDA

#### _Model 1_

In [ ]:
print(len(dataset.labels))
print(len(dataset.images))
#print(len(dataset2.tooth_numbers))

In [ ]:
for i in range(0,8):
    print(i, ":", dataset.labels.count(i))

In [ ]:
plt.hist(dataset.labels)
plt.xlabel('class')
plt.ylabel('count')

#### _Model 2_

In [ ]:
print(len(dataset2.labels))
print(len(dataset2.images))
#print(len(dataset.tooth_numbers))

In [ ]:
for i in range(0,9):
    print(i, ":", dataset2.labels.count(i))

In [ ]:
plt.hist(dataset2.labels)
plt.xlabel('class')
plt.ylabel('count')

# Run Model

#### _Model 1_

In [ ]:
for epoch in range(1, epochs+1):
    train(train_loader, model, 8, optimizer, epoch)
    valid(valid_loader, model, 8, epoch)

In [ ]:
PATH = 'model1.pth'
torch.save(model.state_dict(), PATH)


#### _Model 2_

In [120]:
for epoch in range(1, epochs+1):
    train(train_loader, model2, 9, optimizer, epoch)
    valid(valid_loader, model2, 9, epoch)

<ipython-input-116-12cc02487420>:129: UserWarning: nn.init.kaiming_normal is now deprecated in favor of nn.init.kaiming_normal_.
  init.kaiming_normal(self.fc.weight)
<ipython-input-116-12cc02487420>:133: UserWarning: nn.init.kaiming_normal is now deprecated in favor of nn.init.kaiming_normal_.
  init.kaiming_normal(self.state_dict()[key], mode='fan_out')
  0%|          | 0/366 [00:00<?, ?it/s]C:\Users\admin\.conda\envs\dental1_yolo\lib\site-packages\torch\nn\functional.py:1805: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")
100%|██████████| 366/366 [00:54<00:00,  6.71it/s]


Epoch 1, loss = 0.4680, batch_idx= 366
Epoch: 1 Accuracy of the Train Images: 5068.046174
Confusion Matrix [[18556.   173.]
 [ 4510.   151.]]
Classification Report               precision    recall  f1-score   support

         0.0       0.80      0.99      0.89     18729
         1.0       0.47      0.03      0.06      4661

    accuracy                           0.80     23390
   macro avg       0.64      0.51      0.47     23390
weighted avg       0.74      0.80      0.72     23390



  0%|          | 0/92 [00:00<?, ?it/s]C:\Users\admin\.conda\envs\dental1_yolo\lib\site-packages\torch\nn\functional.py:1805: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")
100%|██████████| 92/92 [00:09<00:00,  9.51it/s]


Epoch: 1 Accuracy of the Valid Images: 5081.019152
Confusion Matrix [[4654.   29.]
 [1146.   19.]]
Classification Report               precision    recall  f1-score   support

         0.0       0.80      0.99      0.89      4683
         1.0       0.40      0.02      0.03      1165

    accuracy                           0.80      5848
   macro avg       0.60      0.51      0.46      5848
weighted avg       0.72      0.80      0.72      5848



  0%|          | 0/366 [00:00<?, ?it/s]C:\Users\admin\.conda\envs\dental1_yolo\lib\site-packages\torch\nn\functional.py:1805: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")
100%|██████████| 366/366 [00:54<00:00,  6.70it/s]


...

Epoch: 99 Accuracy of the Valid Images: 4812.927497
Confusion Matrix [[4431.  252.]
 [ 958.  207.]]
Classification Report               precision    recall  f1-score   support

         0.0       0.82      0.95      0.88      4683
         1.0       0.45      0.18      0.25      1165

    accuracy                           0.79      5848
   macro avg       0.64      0.56      0.57      5848
weighted avg       0.75      0.79      0.76      5848



  0%|          | 0/366 [00:00<?, ?it/s]C:\Users\admin\.conda\envs\dental1_yolo\lib\site-packages\torch\nn\functional.py:1805: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")
100%|██████████| 366/366 [00:54<00:00,  6.76it/s]


Epoch 100, loss = 0.0075, batch_idx= 366
Epoch: 100 Accuracy of the Train Images: 4384.600257
Confusion Matrix [[18702.    27.]
 [   31.  4630.]]
Classification Report               precision    recall  f1-score   support

         0.0       1.00      1.00      1.00     18729
         1.0       0.99      0.99      0.99      4661

    accuracy                           1.00     23390
   macro avg       1.00      1.00      1.00     23390
weighted avg       1.00      1.00      1.00     23390



  0%|          | 0/92 [00:00<?, ?it/s]C:\Users\admin\.conda\envs\dental1_yolo\lib\site-packages\torch\nn\functional.py:1805: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")
100%|██████████| 92/92 [00:09<00:00,  9.70it/s]


Epoch: 100 Accuracy of the Valid Images: 4346.203830
Confusion Matrix [[3949.  734.]
 [ 718.  447.]]
Classification Report               precision    recall  f1-score   support

         0.0       0.85      0.84      0.84      4683
         1.0       0.38      0.38      0.38      1165

    accuracy                           0.75      5848
   macro avg       0.61      0.61      0.61      5848
weighted avg       0.75      0.75      0.75      5848



In [122]:
PATH = 'model2.pth'
torch.save(model2.state_dict(), PATH)


<ipython-input-116-12cc02487420>:129: UserWarning: nn.init.kaiming_normal is now deprecated in favor of nn.init.kaiming_normal_.
  init.kaiming_normal(self.fc.weight)
<ipython-input-116-12cc02487420>:133: UserWarning: nn.init.kaiming_normal is now deprecated in favor of nn.init.kaiming_normal_.
  init.kaiming_normal(self.state_dict()[key], mode='fan_out')


ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (avgpool): AvgPool2d(kernel_size=7, stride=7, padding=0)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (cbam): CBAM(
        (ChannelGate): ChannelGate(
          (mlp): Sequential(
            (0): Flatten()
            (1): Linear(in_features=64, out_features=4, bias=True)
          